In [855]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer

In [856]:
df = pd.read_csv('bail_cleaned_data.csv')
# Drops the rows where the bond amt is $0
df = df[df.bond_amt != 0.0]
print(df.shape)
# Drops the outlier rows over 150,000
# df = df[df.bond_amt < 150000]
df.dtypes

(152333, 6)


Unnamed: 0       int64
judge           object
prior_cases    float64
bond_amt       float64
race            object
plea_orcs      float64
dtype: object

In [857]:
# Everything besides the bond amt column
bond = df[['judge', 'plea_orcs', 'race', 'prior_cases']]
# Bond amt column
bond_labels = df['bond_amt'].copy()

In [858]:
bond_num = list(bond.columns[bond.dtypes == 'float64'])
bond_cat = list(bond.columns[bond.dtypes == 'object'])

# Pipelines
num_pipe = Pipeline([
    ('std', StandardScaler())
])

cat_pipe = Pipeline([
    ('ohe', OneHotEncoder())
])

In [907]:
bond_num = list(bond.columns[bond.dtypes == 'float64'])
bond_cat = list(bond.columns[bond.dtypes == 'object'])

full_pipeline = ColumnTransformer(transformers=[
    ('num', num_pipe, bond_num),
    ('cat', cat_pipe, bond_cat)
])

bond_prep = full_pipeline.fit_transform()

TypeError: fit_transform() missing 1 required positional argument: 'X'

In [872]:
X_train, X_test, y_train, y_test = train_test_split(bond_prep, bond_labels, test_size=.2)

In [889]:
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)

DecisionTreeRegressor()

In [901]:
# 'plea_orcs', 'judge', 'race', 'prior_cases'
d = {'plea_orcs':[2913.11], 'judge':['TERRY_SJ'], 'race':['WHITE'], 'prior_cases':[0]}
user_data = pd.DataFrame.from_dict(d)
y_pred = tree.predict(X_test)
y_pred

array([40500.        , 34656.25      ,  4360.        , ...,
        8214.28571429, 21875.        ,  6000.        ])

In [883]:
some_data = df.iloc[:5]
some_labels = y_train.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

ValueError: X has 6 features, but ColumnTransformer is expecting 4 features as input.

In [798]:
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth': [3,10], 'min_samples_leaf': [2, 3, 4]}


grid = GridSearchCV(DecisionTreeRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid.fit(X_train, y_train)

ValueError: could not convert string to float: 'FUERST_NA'